In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from requests import get

# SCRAPING CON APIs de "BIPICAR":
Obtener scraping de coches de alquiler por meses para tener comparación de precios respecto a nuestro dataset de ofertas de coches de segunda mano. Mediante APIs

Nuestro objetivo es extraer los datos mediante la API: https://bipicar.com/api-region/specificvehicleplans/62f0b59c9629a6b2b61b1e7d/catalog-item?region=ES .

Para lo cual necesitamos primero los códigos que corresponden con los diferentes coches para poder ir modificando la API e ir obteniendo los datos para cada coche.

## 1 OBTENIENDO CÓDIGOS DE URLS, IDENTIFICADORES DE LOS COCHES DE ALQUILER:

In [ ]:
url = "https://bipicar.com/es/es/coches"
response = get(url)
response

<Response [200]>

In [ ]:
# También podemos hacerlo pidiéndole el idioma devuelto:
# Nos devuelve una petición exitosa (código 200)
# Para este proyecto, en principio, nos da igual que esté en español.
get(url, headers = {"Accept-Language":"es"})

<Response [200]>

In [ ]:
# Obtenemos la respuesta en texto:
response.text

'<!doctype html><html data-n-head-ssr lang="es-ES" data-n-head="%7B%22lang%22:%7B%22ssr%22:%22es-ES%22%7D%7D"><head><meta data-n-head="ssr" charset="utf-8"><meta data-n-head="ssr" name="viewport" content="width=device-width,initial-scale=1,user-scalable=no"><meta data-n-head="ssr" data-hid="i18n-og" property="og:locale" content="es_ES"><meta data-n-head="ssr" data-hid="i18n-og-alt-de-DE" property="og:locale:alternate" content="de_DE"><meta data-n-head="ssr" data-hid="i18n-og-alt-en-GB" property="og:locale:alternate" content="en_GB"><meta data-n-head="ssr" data-hid="i18n-og-alt-nl-NL" property="og:locale:alternate" content="nl_NL"><meta data-n-head="ssr" data-hid="i18n-og-alt-it-IT" property="og:locale:alternate" content="it_IT"><meta data-n-head="ssr" data-hid="i18n-og-alt-fr-FR" property="og:locale:alternate" content="fr_FR"><meta data-n-head="ssr" name="description" data-hid="description" content="Coches por suscripciÃ³n flexible. Desde 1 mes hasta 3, 6, 12,18, 24 meses. La mejor opc

In [ ]:
# obtenemos el código html de la página web en texto:
sopa = BeautifulSoup(response.text, "html.parser")
sopa

<!DOCTYPE html>
<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22es-ES%22%7D%7D" data-n-head-ssr="" lang="es-ES"><head><meta charset="utf-8" data-n-head="ssr"/><meta content="width=device-width,initial-scale=1,user-scalable=no" data-n-head="ssr" name="viewport"/><meta content="es_ES" data-hid="i18n-og" data-n-head="ssr" property="og:locale"/><meta content="de_DE" data-hid="i18n-og-alt-de-DE" data-n-head="ssr" property="og:locale:alternate"/><meta content="en_GB" data-hid="i18n-og-alt-en-GB" data-n-head="ssr" property="og:locale:alternate"/><meta content="nl_NL" data-hid="i18n-og-alt-nl-NL" data-n-head="ssr" property="og:locale:alternate"/><meta content="it_IT" data-hid="i18n-og-alt-it-IT" data-n-head="ssr" property="og:locale:alternate"/><meta content="fr_FR" data-hid="i18n-og-alt-fr-FR" data-n-head="ssr" property="og:locale:alternate"/><meta content="Coches por suscripciÃ³n flexible. Desde 1 mes hasta 3, 6, 12,18, 24 meses. La mejor opciÃ³n del mercado para disponer de un vehÃ­culo cu

In [ ]:
# Obtenenmos el html de la página web, pero ordenado y más claro:
# Aquí ya podemos localizar la parte que nos interesa y localizar posibles etiquetas.
# También desde la página, con inspeccionar, podemos localizar lo interesante, facilmente.
print(sopa.prettify())

<!DOCTYPE html>
<html data-n-head="%7B%22lang%22:%7B%22ssr%22:%22es-ES%22%7D%7D" data-n-head-ssr="" lang="es-ES">
 <head>
  <meta charset="utf-8" data-n-head="ssr"/>
  <meta content="width=device-width,initial-scale=1,user-scalable=no" data-n-head="ssr" name="viewport"/>
  <meta content="es_ES" data-hid="i18n-og" data-n-head="ssr" property="og:locale"/>
  <meta content="de_DE" data-hid="i18n-og-alt-de-DE" data-n-head="ssr" property="og:locale:alternate"/>
  <meta content="en_GB" data-hid="i18n-og-alt-en-GB" data-n-head="ssr" property="og:locale:alternate"/>
  <meta content="nl_NL" data-hid="i18n-og-alt-nl-NL" data-n-head="ssr" property="og:locale:alternate"/>
  <meta content="it_IT" data-hid="i18n-og-alt-it-IT" data-n-head="ssr" property="og:locale:alternate"/>
  <meta content="fr_FR" data-hid="i18n-og-alt-fr-FR" data-n-head="ssr" property="og:locale:alternate"/>
  <meta content="Coches por suscripciÃ³n flexible. Desde 1 mes hasta 3, 6, 12,18, 24 meses. La mejor opciÃ³n del mercado par

### - find()

Localizar el identificador url de un coche:

In [ ]:
# Encontramos la cadena donde se encuentra el código para un coche:
sopa1 = sopa.find("span", class_ = "row equal").find("div", class_ = "flotaItem col-lg-3 col-md-6 col-sm-12").find("a").get("href")
sopa1

'/es/es/coches/fiat/500/6447e0554316d1f50ef21560'

In [ ]:
# Lo mismo pero usando otro método final de "href":
sopa_url = sopa.find("span", class_ = "row equal").find("div", class_ = "flotaItem col-lg-3 col-md-6 col-sm-12").find("a")["href"]
sopa_url

'/es/es/coches/fiat/500/6447e0554316d1f50ef21560'

In [ ]:
# Separamos la cadena que contiene el código url en elementos para poder coger solo el código:
sopa_url2 = sopa_url.split("/")
sopa_url2

['', 'es', 'es', 'coches', 'fiat', '500', '6447e0554316d1f50ef21560']

In [ ]:
# Nos quedamos solo con el código identificador del coche:
sopa_url2[-1]

'6447e0554316d1f50ef21560'

### - fin_all()

Localizar todos los identificadores url de los coches:
Una vez tenemos localizado un identificador, vamos a extraer todos:

In [ ]:
# Localizamos todos con find_all().
sopa_url_all = sopa.find_all("div", class_ = "flotaItem col-lg-3 col-md-6 col-sm-12")
sopa_url_all[0]

<div class="flotaItem col-lg-3 col-md-6 col-sm-12" data-cy="coche0" data-plan="coche-6447e0554316d1f50ef21560"><div><a class="flotaItem__container" href="/es/es/coches/fiat/500/6447e0554316d1f50ef21560"><div class="flotaItem__info"><div class="model"><h5 class="bold mainVehicle">Fiat 500</h5><p class="category mb-xs-2">CityCar</p></div><div><p class="flotaItem__price"><span>Desde</span>299â¬<span>/mes</span></p></div></div><div class="flotaItem__img"><img alt="VehÃ­culo Fiat 500 CityCar" class="lazyload" data-src="https://cdn.bipicar.com/specificvehicleplans/60a24becfb2bb0008dc6250c/vehicle-images/6246b5e60dc7c5c7de220ea4_2022_04_01_10_20_54.png" src="/_nuxt/img/e929b7d.jpg"/><div class="badges"></div></div><div class="flotaItem__flexibility"><p>Opciones<span>Sin permanencia, 3, 6, 24 meses</span></p></div></a></div></div>

#### Extraer identificadores:

In [ ]:
# ¿Cuantos identificadores o coches tenemos?
print(f"Tenemos {len(sopa_url_all)} identificadores url o coches")

Tenemos 50 identificadores url o coches


In [ ]:
# Extraemos todos los idents y los guardamos en una lista:
codigos = []

for i in sopa_url_all:
  url_codigo = i.find("a")["href"].split("/")[-1] #separamos y nos quedamos solo con el ident.
  codigos.append(url_codigo)

codigos

['6447e0554316d1f50ef21560',
 '6022938c017e1200aabb9d3c',
 '64acff87d4e19fd75aefb2c4',
 '63bbdc9a3737a8548e305e17',
 '60b8ae4d12c0270084627399',
 '5f6dc6cdeaa82900b344d16e',
 '64a7c2cf0a04165f671c20d5',
 '627bc2267596821e31363b97',
 '62a99af900242bdf32be10a4',
 '6320a5c052c92cf5d9cf5d5d',
 '6495744daaf5be24cbd58575',
 '64259d9d13f6b57e88b25f3d',
 '63fc9ea7a42a4f0be90c7cb7',
 '6093f90d320cff0086bc58eb',
 '646396e222f660a49c811416',
 '64abd903bd27700df7440713',
 '643676eba001cd07bfea3cca',
 '633c56a7142c188e52f7de49',
 '6033a968c37ea000a54e2d27',
 '5f3a56b7ae2c8000aa805225',
 '64aba6057b7a5efadd9e66b9',
 '64ad1da4d4589dd47266223f',
 '615dade0843dbc008731ed0f',
 '60a273c8d988e1008d1dcf68',
 '6422cb3253f7b724c6617840',
 '5ee7651c5f5120009d6fd247',
 '62cc0fb0ccc7099ce70886f7',
 '608a5a54e19d910099286607',
 '64199596b56a2f164ea32918',
 '636263cf658ae054644097c9',
 '626aa2ecf42154ec4ec15cd7',
 '6491b97f940139710c1dc6fb',
 '639716dfb59542c89a743ef4',
 '6113c67b5542c80084b953bc',
 '6437d1b589f3

### Links APIs con sus correspondientes identificadores:

Tenemos los identificadores de todos los coches. Vamos a formar sus links url APIS para poder pedir la información a cada uno:

In [ ]:
# Partimos de la API principal, proporcionada de antemano.
print("Longitud de códigos es: ",len(codigos))
api = "https://bipicar.com/api-region/specificvehicleplans/62f0b59c9629a6b2b61b1e7d/catalog-item?region=ES"
print("API: ",api)

Longitud de códigos es:  50
API:  https://bipicar.com/api-region/specificvehicleplans/62f0b59c9629a6b2b61b1e7d/catalog-item?region=ES


In [ ]:
# Creamos una lista con todos los links APIs con sus identificadores,
# reemplazando el ident original por cada uno de los ident de la lista codigos:

lista_urls_con_identificador = []

for ident in codigos:
  url_con_ident_unico = api.replace((api.split("/")[-2]),ident) # Cambiamos ident original por cada uno de lista codigos.
  lista_urls_con_identificador.append(url_con_ident_unico) # A lista de APIs le añadimos el nuevo link, ya modificado.


lista_urls_con_identificador # Mostramos lista de APIs con identificadores únicos.

['https://bipicar.com/api-region/specificvehicleplans/6447e0554316d1f50ef21560/catalog-item?region=ES',
 'https://bipicar.com/api-region/specificvehicleplans/6022938c017e1200aabb9d3c/catalog-item?region=ES',
 'https://bipicar.com/api-region/specificvehicleplans/64acff87d4e19fd75aefb2c4/catalog-item?region=ES',
 'https://bipicar.com/api-region/specificvehicleplans/63bbdc9a3737a8548e305e17/catalog-item?region=ES',
 'https://bipicar.com/api-region/specificvehicleplans/60b8ae4d12c0270084627399/catalog-item?region=ES',
 'https://bipicar.com/api-region/specificvehicleplans/5f6dc6cdeaa82900b344d16e/catalog-item?region=ES',
 'https://bipicar.com/api-region/specificvehicleplans/64a7c2cf0a04165f671c20d5/catalog-item?region=ES',
 'https://bipicar.com/api-region/specificvehicleplans/627bc2267596821e31363b97/catalog-item?region=ES',
 'https://bipicar.com/api-region/specificvehicleplans/62a99af900242bdf32be10a4/catalog-item?region=ES',
 'https://bipicar.com/api-region/specificvehicleplans/6320a5c052

YA TENEMOS TODAS LAS APIs!

## 2 DATAFRAME MEDIANTE APIS:

Vamos a ir extrayendo los datos de las APIs para crear un dataframe.

Creamos una función que lo hace automáticamente pasandole una lista de APIs

In [ ]:
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests


In [ ]:
# Esto es lo que os decía antes chicos
    # response = requests.get(url, params={'q': 'python'})
    # response = response.json()
    # df_response = pd.json_normalize(response)


    # import requests

    # parametros = {'clave1': 'valor1', 'clave2': ['val1', 'val2']}
    # r = requests.get('http://unblog.xyz', params=parametros)



In [ ]:
# Obtenemos un df con una sola fila correspondiente a la peticion de una API y obteniendo una fila de una oferta de un coche.

response = requests.get("https://bipicar.com/api-region/specificvehicleplans/6349514c58fbd070efe95edf/catalog-item?region=ES",
                        params={'q': 'python'})
response = response.json()
df_response = pd.json_normalize(response)
df_response

,_id,mainVehicle,vehicleImg,secondaryImages,category,features,equipmentHtml,mainVehicleBrand,mainVehicleModel,infoHtml,...,isPurchasable,specificvehicleplanversion,typedFeatures.transmission,typedFeatures.seats,typedFeatures.fuelType,typedFeatures.year,typedFeatures.modelVersion,typedFeatures.status,typedFeatures.powerCV,typedFeatures.engineCapacityCC
0,6349514c58fbd070efe95edf,Fiat Tipo SW,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,SW,[],<h2>Multimedia</h2><ul><li>Cuadro de instrumen...,Fiat,Tipo SW,None,...,False,64808874e851ea4f49467a94,[MANUAL],[5],[DIESEL],[2022],Mjet Life,NEW,95,1300


In [ ]:
# Comprimimos para que sea más comprensible el código final:
    # response1 = requests.get(lista_urls_con_identificador[0], params={'q': 'python'})
    # response1 = response1.json()
    # df_response1 = pd.json_normalize(response1)

# Lo convertimos en esto:

  # df_response1 = pd.json_normalize(requests.get(lista_urls_con_identificador[0], params={'q': 'python'}).json()) #Comprimido en una línea
  # df_response1

### 2.1 Función Crear dataframes a partir de APIs y concatenar en uno solo!

In [ ]:
#Creamos funcion para:
  # 1 Crear dataframes a partir de APIs
  # 2 Estos dataframes unirnos en un solo dataframe llamado "df_apis"

# df_response_base = pd.json_normalize(requests.get(lista_urls_con_identificador[0], params={'q': 'python'}).json()) #Comprimido en una línea

def crear_concatenar_dataframes_con_apis(lista_apis):
  global df_apis
  df_apis = pd.json_normalize(requests.get(lista_urls_con_identificador[0], params={'q': 'python'}).json()) #Comprimido en una línea

  for i in range (1, len(lista_apis)): # desde 1 porque la primera api ya es el propio df_apis
    if i <= len(lista_apis):
      df1 = pd.json_normalize(requests.get(lista_apis[i], params={'q': 'python'}).json())
      df_apis = pd.concat([df_apis, df1])
  return df_apis



In [ ]:
# Simplemente llamamos a la función, pasandole lista de APIs y obtenemos el dataframe completo de todas las APIs:

crear_concatenar_dataframes_con_apis(lista_urls_con_identificador)

,_id,mainVehicle,vehicleImg,secondaryImages,category,features,equipmentHtml,mainVehicleBrand,mainVehicleModel,infoHtml,...,contract.kmIncluded,contract.extraKmPrice,typedFeatures.engineCapacityCC,badge.defaultLanguage,badge.color,badge.languages,campaignImg,offerPrice,offerBadge.badgeText,offerBadge.color
0,6447e0554316d1f50ef21560,Fiat 500,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,CityCar,[],<h2>Multimedia</h2><ul><li>Bluetooth</li><li>V...,Fiat,500,<p>Coche de ocasión certificado y verificado p...,...,800,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,6022938c017e1200aabb9d3c,Peugeot 208,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,Utilitario,"[85Cv de potencia, Color disponible - Blanco]",<h2>Multimedia</h2><ul><li>Ordenador de abordo...,Peugeot,208,<p>Coche de ocasión certificado y verificado p...,...,800,0.15,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,64acff87d4e19fd75aefb2c4,Ford Ecosport,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,B-SUV,[],<h2>Multimedia</h2><ul><li>Bluetooth</li><li>6...,Ford,Ecosport,<p>Coche de ocasión certificado y verificado p...,...,800,0.15,999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,63bbdc9a3737a8548e305e17,Seat Ibiza,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,Utilitario,"[115 Cv, 5 Plazas, Manual, Diésel, Seminuevo]",<h2>Multimedia</h2><ul><li>Ordenador de abordo...,Seat,Ibiza,<p>Coche de ocasión certificado y verificado p...,...,800,0.15,1598.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,60b8ae4d12c0270084627399,Peugeot 3008,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,C-SUV,"[130 Cv, Acabado GT Line]",<h2>Multimedia</h2><ul><li>Volante compacto mu...,Peugeot,3008,<p>Coche de seminuevo certificado y verificado...,...,800,0.15,1499.0,ES,#666665,"[{'key': 'ES', 'badgeText': 'AUTOMÁTICO', '_id...",NaN,NaN,NaN,NaN
0,5f6dc6cdeaa82900b344d16e,Lexus UX,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,C-SUV,"[120 Cv, 5 Plazas, Manual, Diésel, Blanco Gris]",<h2>Multimedia</h2><ul><li>Bluetooth (Teléfono...,Lexus,UX,None,...,800,0.20,2000.0,ES,#0bce92,"[{'key': 'ES', 'badgeText': 'HÍBRIDO', '_id': ...",https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,575.0,OFERTA,NaN
0,64a7c2cf0a04165f671c20d5,Toyota C-HR,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,C-SUV,[],<h2>Multimedia</h2><ul><li>Volante multifunció...,Toyota,C-HR,NaN,...,800,0.15,1987.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,627bc2267596821e31363b97,DS DS7,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,D-SUV,[],<h2>Multimedia</h2><ul><li>Ordenador de a bord...,DS,DS7,<p>Coche de ocasión certificado y verificado p...,...,800,0.15,1199.0,ES,#666665,"[{'key': 'ES', 'badgeText': 'AUTOMÁTICO', '_id...",NaN,NaN,NaN,NaN
0,62a99af900242bdf32be10a4,Citroen C3,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,Utilitario,[],<h2>Multimedia</h2><ul><li>Bluetooth</li><li>C...,Citroen,C3,<p>Coche de ocasión certificado y verificado p...,...,800,0.15,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,6320a5c052c92cf5d9cf5d5d,Renault Megane,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,Compacto,[],<h2>Multimedia</h2><ul><li>Unidad de conectivi...,Renault,Megane,None,...,800,0.15,NaN,ES,#42a5f5,"[{'key': 'ES', 'badgeText': 'ELÉCTRICO', '_id'...",NaN,NaN,NaN,NaN


Ya tenemos nuestro dataframe con los coches en alquiler, que se llama "df_apis"

In [ ]:
# Forma de nuestro df:

print(f"El df contiene {df_apis.shape[0]} filas o coches en renting y {df_apis.shape[1]} columnas.")

El df contiene 50 filas o coches en renting y 46 columnas.


In [ ]:
df_apis.head()

,_id,mainVehicle,vehicleImg,secondaryImages,category,features,equipmentHtml,mainVehicleBrand,mainVehicleModel,infoHtml,...,contract.kmIncluded,contract.extraKmPrice,typedFeatures.engineCapacityCC,badge.defaultLanguage,badge.color,badge.languages,campaignImg,offerPrice,offerBadge.badgeText,offerBadge.color
0,6447e0554316d1f50ef21560,Fiat 500,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,CityCar,[],<h2>Multimedia</h2><ul><li>Bluetooth</li><li>V...,Fiat,500,<p>Coche de ocasión certificado y verificado p...,...,800,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,6022938c017e1200aabb9d3c,Peugeot 208,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,Utilitario,"[85Cv de potencia, Color disponible - Blanco]",<h2>Multimedia</h2><ul><li>Ordenador de abordo...,Peugeot,208,<p>Coche de ocasión certificado y verificado p...,...,800,0.15,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,64acff87d4e19fd75aefb2c4,Ford Ecosport,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,B-SUV,[],<h2>Multimedia</h2><ul><li>Bluetooth</li><li>6...,Ford,Ecosport,<p>Coche de ocasión certificado y verificado p...,...,800,0.15,999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,63bbdc9a3737a8548e305e17,Seat Ibiza,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,Utilitario,"[115 Cv, 5 Plazas, Manual, Diésel, Seminuevo]",<h2>Multimedia</h2><ul><li>Ordenador de abordo...,Seat,Ibiza,<p>Coche de ocasión certificado y verificado p...,...,800,0.15,1598.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,60b8ae4d12c0270084627399,Peugeot 3008,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,C-SUV,"[130 Cv, Acabado GT Line]",<h2>Multimedia</h2><ul><li>Volante compacto mu...,Peugeot,3008,<p>Coche de seminuevo certificado y verificado...,...,800,0.15,1499.0,ES,#666665,"[{'key': 'ES', 'badgeText': 'AUTOMÁTICO', '_id...",NaN,NaN,NaN,NaN


In [ ]:
# Vamos a renombrar el df_apis como df a secas porque es más comodo de utilizar:
df = df_apis.copy() # no es necesario crear copia, no vamos a usarlo más como df_apis

# Comprobamos que este bien:
df.head()

,_id,mainVehicle,vehicleImg,secondaryImages,category,features,equipmentHtml,mainVehicleBrand,mainVehicleModel,infoHtml,...,contract.kmIncluded,contract.extraKmPrice,typedFeatures.engineCapacityCC,badge.defaultLanguage,badge.color,badge.languages,campaignImg,offerPrice,offerBadge.badgeText,offerBadge.color
0,6447e0554316d1f50ef21560,Fiat 500,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,CityCar,[],<h2>Multimedia</h2><ul><li>Bluetooth</li><li>V...,Fiat,500,<p>Coche de ocasión certificado y verificado p...,...,800,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,6022938c017e1200aabb9d3c,Peugeot 208,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,Utilitario,"[85Cv de potencia, Color disponible - Blanco]",<h2>Multimedia</h2><ul><li>Ordenador de abordo...,Peugeot,208,<p>Coche de ocasión certificado y verificado p...,...,800,0.15,1200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,64acff87d4e19fd75aefb2c4,Ford Ecosport,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,B-SUV,[],<h2>Multimedia</h2><ul><li>Bluetooth</li><li>6...,Ford,Ecosport,<p>Coche de ocasión certificado y verificado p...,...,800,0.15,999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,63bbdc9a3737a8548e305e17,Seat Ibiza,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,Utilitario,"[115 Cv, 5 Plazas, Manual, Diésel, Seminuevo]",<h2>Multimedia</h2><ul><li>Ordenador de abordo...,Seat,Ibiza,<p>Coche de ocasión certificado y verificado p...,...,800,0.15,1598.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,60b8ae4d12c0270084627399,Peugeot 3008,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,[{'url': 'https://bipi-pro.s3.eu-west-1.amazon...,C-SUV,"[130 Cv, Acabado GT Line]",<h2>Multimedia</h2><ul><li>Volante compacto mu...,Peugeot,3008,<p>Coche de seminuevo certificado y verificado...,...,800,0.15,1499.0,ES,#666665,"[{'key': 'ES', 'badgeText': 'AUTOMÁTICO', '_id...",NaN,NaN,NaN,NaN


## 3 ETL

### Adecuar columnas:

# Tenemos 46 columnas, pero no todas nos aportan información relevante para nuestro proyecto.

# Lo primero es quitar las columnas que no nos interesan:
---

Podemos eliminar de nuestro df las columnas:
*  "Unnamed" (Valor único "0"),
*  "_id" (Códigos identificadores de url),
*  "secondaryImages" (Con una imagen principal es suficiente, col "vehicleImg"),
*  "equipmentHtml" (Código html, no aporta información),
*  "region" (Valor único "Es" de España),
*  "hasFlexibleadvantages" (Valor único "True"),
*  "flexibleadvantages" (Flexibilidad pago financiado con bancos, no nos interesa),
*  "subscriptionType" (Valor único "Month"),
*  "showBadge" (Valores "True" o "False", tarjeta para mostrar),
*  "catalogIndex" (Índice de catalogo),
*  "specificvehicleplanversion" (Código identificador),
*  "isOnOffer" (Está en oferta, irrelevante, solo queremos precio para comparar nuestro df segunda mano),
*  "allowRemoveExcess" (Si se permite descuento),
*  "removeExcessPrice" (Código descuento para aplicar),
*  "franchiseValue" (Algún tipo de franquicia),
*  "reducedFranchiseValue" (Valores 0 o Nan),
*  "sharedUserPrice (No aporta información),
*  "commitmentMonths" (Algo de meses, no nos interesa),
*  "addons" (Complementos para el coche para el alquiler, no interesa),
*  "hasSharedSubscriptionsEnabled" (TIENE LAS SUSCRIPCIONES COMPARTIDAS HABILITADAS),
*  "contract.kmIncluded" (Km de contrato incluido, no nos interesa),
*  "contract.extraKmPrice" (No nos interesa),
*  #### Está ahora no aparece: "campaignImg" (Imagen de oferta, no interesa),
*  "offerPrice" (No interesa),
*  "offerBadge.color" (Código de color),
*  "offerBadge.badgeText" (Texto de una oferta),
*  "badge.defaultLanguage" (Valor único España, y Nan),
*  "badge.color" (Código color),
*  "badge.languages" (Código etiqueta)

In [ ]:
# Las columnas que no nos interesan las pasamos a lista:
lista_malas = ["Unnamed","_id","secondaryImages","equipmentHtml","region","hasFlexibleadvantages","flexibleadvantages","subscriptionType",
               "showBadge","catalogIndex","specificvehicleplanversion","isOnOffer","allowRemoveExcess","removeExcessPrice","franchiseValue","reducedFranchiseValue",
               "sharedUserPrice","commitmentMonths","addons","hasSharedSubscriptionsEnabled","contract.kmIncluded","contract.extraKmPrice",
               "offerPrice","offerBadge.color","offerBadge.badgeText","badge.defaultLanguage","badge.color","badge.languages", 'campaignImg']



In [ ]:
# Eliminamos columnas no interesantes del df:
df.drop(lista_malas[1:], axis=1, inplace=True) # la primera columna es indice y no lo detecta como colunma, la quitamos.

# ok, tenemos solo las cols interesantes en el df, comprobamos:
df.head(1)

,mainVehicle,vehicleImg,category,features,mainVehicleBrand,mainVehicleModel,infoHtml,groupName,isPurchasable,price,typedFeatures.transmission,typedFeatures.seats,typedFeatures.fuelType,typedFeatures.year,typedFeatures.modelVersion,typedFeatures.status,typedFeatures.powerCV,typedFeatures.engineCapacityCC
0,Fiat 500,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,CityCar,[],Fiat,500,<p>Coche de ocasión certificado y verificado p...,1,True,299,[MANUAL],[4],[GAS],[2018],1.2 Lounge 69CV,OPPORTUNITY,69,NaN


In [ ]:
# Una vez tenemos las columnas interesantes en el df:
  # Traducimos los nombres de las columnas:


!pip install deep_translator
from deep_translator import GoogleTranslator

# - - Esta parte del código traduce únicamente un texto plano
# traductor = GoogleTranslator(source='es', target='en') # target es idioma al que queremos traducir.
# resultado = traductor.translate("La educación es el arma más poderosa para cambiar al mundo") # cadena para traducir.
# print(resultado)
#-----------------------------------------------------------------------------


# - - Esta parte del código traduce un archivo, esta comentada porque si el archivo no existe dará error.
#translated = GoogleTranslator(source='en', target='es').translate_file(r'C:\Users\Tecsify.txt')
#print(translated)
#-----------------------------------------------------------------------------


# - - Esta parte del código traduce una lista de palabras por separado sin que se vean afectadas por el contexto de la oración.

traductor = GoogleTranslator(source='auto', target='es') # source define el lenguaje del texto, target es idioma respuesta traducción.
lista_traducidas = traductor.translate_batch(list(df.columns)) # lista para traducir
lista_traducidas
#-----------------------------------------------------------------------------

['vehículoprincipal',
 'vehículoImg',
 'categoría',
 'características',
 'principalVehículoMarca',
 'principalVehículoModelo',
 'infoHtml',
 'Nombre del grupo',
 'se puede comprar',
 'precio',
 'typedFeatures.transmisión',
 'typedFeatures.asientos',
 'typedFeatures.fuelType',
 'typedFeatures.año',
 'typedFeatures.modelVersion',
 'typedFeatures.status',
 'typedFeatures.powerCV',
 'typedFeatures.engineCapacityCC']

In [ ]:
# Cambiamos las columnas en inglés por columnas en español:

df.columns = lista_traducidas

# Comprobamos:
df.head(1)

,vehículoprincipal,vehículoImg,categoría,características,principalVehículoMarca,principalVehículoModelo,infoHtml,Nombre del grupo,se puede comprar,precio,typedFeatures.transmisión,typedFeatures.asientos,typedFeatures.fuelType,typedFeatures.año,typedFeatures.modelVersion,typedFeatures.status,typedFeatures.powerCV,typedFeatures.engineCapacityCC
0,Fiat 500,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,CityCar,[],Fiat,500,<p>Coche de ocasión certificado y verificado p...,1,True,299,[MANUAL],[4],[GAS],[2018],1.2 Lounge 69CV,OPPORTUNITY,69,NaN


In [ ]:
# Vamos a renombrar las columnas para que sean más intuitivas y visualmente más cómodas de trabajar:
# Nuevos nombres de columnas:

df.columns = ['Vehiculo', 'Url_foto', 'categoría', 'Caracteristicas', 'Marca', 'Modelo', 'Kms', 'Grupo', 'Puede_comprarse',
               'Precio', 'Cambio', 'Plazas', 'Combustible', 'Año', 'Version', 'Estado', 'Potencia', 'Centimetros_cubicos']

df.head(1)

,Vehiculo,Url_foto,categoría,Caracteristicas,Marca,Modelo,Kms,Grupo,Puede_comprarse,Precio,Cambio,Plazas,Combustible,Año,Version,Estado,Potencia,Centimetros_cubicos
0,Fiat 500,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,CityCar,[],Fiat,500,<p>Coche de ocasión certificado y verificado p...,1,True,299,[MANUAL],[4],[GAS],[2018],1.2 Lounge 69CV,OPPORTUNITY,69,NaN


In [ ]:
# reseteamos el índice:

df.reset_index(drop=True, inplace=True)
df.head(3)

,Vehiculo,Url_foto,categoría,Caracteristicas,Marca,Modelo,Kms,Grupo,Puede_comprarse,Precio,Cambio,Plazas,Combustible,Año,Version,Estado,Potencia,Centimetros_cubicos
0,Fiat 500,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,CityCar,[],Fiat,500,<p>Coche de ocasión certificado y verificado p...,1,True,299,[MANUAL],[4],[GAS],[2018],1.2 Lounge 69CV,OPPORTUNITY,69,NaN
1,Peugeot 208,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,Utilitario,"[85Cv de potencia, Color disponible - Blanco]",Peugeot,208,<p>Coche de ocasión certificado y verificado p...,1,True,319,[MANUAL],[5],[GAS],[2018],1.2 Puretech Active,OPPORTUNITY,82,1200.0
2,Ford Ecosport,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,B-SUV,[],Ford,Ecosport,<p>Coche de ocasión certificado y verificado p...,1,True,369,[MANUAL],[5],[GAS],[2019],1.0T EcoBoost,OPPORTUNITY,125,999.0


In [ ]:
# Cambiamos el orden de las columnas para adecuar el df a nuestro proyecto y a los dfs de ofertas de coches:
# Vehiculo,	Url_foto,	categoría,	Caracteristicas, Grupo,	Puede_comprarse,	Plazas, Estado,	Centimetros_cubicos.
df = df[['Marca', 'Modelo', 'Version', 'Precio', 'Combustible', 'Año', 'Kms', 'Potencia',	'Cambio', 'Vehiculo',	'Url_foto',	'categoría',
         'Caracteristicas', 'Grupo',	'Puede_comprarse',	'Plazas', 'Estado',	'Centimetros_cubicos']]
df.head(3)

,Marca,Modelo,Version,Precio,Combustible,Año,Kms,Potencia,Cambio,Vehiculo,Url_foto,categoría,Caracteristicas,Grupo,Puede_comprarse,Plazas,Estado,Centimetros_cubicos
0,Fiat,500,1.2 Lounge 69CV,299,[GAS],[2018],<p>Coche de ocasión certificado y verificado p...,69,[MANUAL],Fiat 500,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,CityCar,[],1,True,[4],OPPORTUNITY,NaN
1,Peugeot,208,1.2 Puretech Active,319,[GAS],[2018],<p>Coche de ocasión certificado y verificado p...,82,[MANUAL],Peugeot 208,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,Utilitario,"[85Cv de potencia, Color disponible - Blanco]",1,True,[5],OPPORTUNITY,1200.0
2,Ford,Ecosport,1.0T EcoBoost,369,[GAS],[2019],<p>Coche de ocasión certificado y verificado p...,125,[MANUAL],Ford Ecosport,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,B-SUV,[],1,True,[5],OPPORTUNITY,999.0


En nuestro df de ofertas de coches, tenemos otras columnas que en este df no aparecen, pero no importa. Estos coches solo son para comparar los precios de algunos coches y hacernos una idea de si los precios de los coches que vamos a comprar de segunda mano están adsequibles o no.

Adecuamos el tipo de datos de las columnas:

In [ ]:
# tipos de datos que tienen las columnas:

df.dtypes

Marca                   object
Modelo                  object
Version                 object
Precio                   int64
Combustible             object
Año                     object
Kms                     object
Potencia                 int64
Cambio                  object
Vehiculo                object
Url_foto                object
categoría               object
Caracteristicas         object
Grupo                    int64
Puede_comprarse           bool
Plazas                  object
Estado                  object
Centimetros_cubicos    float64
dtype: object

In [ ]:
# Cambiamos tipo de dato de "Grupo" a catégorico por si más adelante tuvieramos problemas
df['Grupo'] = df['Grupo'].astype(object)
# Columna Kms la cambiaremos más adelante, cuando la limpiemos de strings, a intero.

In [ ]:
# Queremos sacar los valores únicos que tiene cada columna, en busca de errores, pero los valores de "Combustible"
# están en una lista. Vamos a sacarlos y a dejarlos como cadena para evitar el error que nos produce cuando preguntamos
# por sus valores únicos:

df['Combustible'] = df['Combustible'].apply(lambda x: x[0])
df['Combustible'].head(5)

0       GAS
1       GAS
2       GAS
3    DIESEL
4    DIESEL
Name: Combustible, dtype: object

In [ ]:
df.head(1)

,Marca,Modelo,Version,Precio,Combustible,Año,Kms,Potencia,Cambio,Vehiculo,Url_foto,categoría,Caracteristicas,Grupo,Puede_comprarse,Plazas,Estado,Centimetros_cubicos
0,Fiat,500,1.2 Lounge 69CV,299,GAS,[2018],<p>Coche de ocasión certificado y verificado p...,69,[MANUAL],Fiat 500,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,CityCar,[],1,True,[4],OPPORTUNITY,NaN


In [ ]:
# Ocurre lo mismo para otras columnas, vamos a sacar estos valores fuera de las listas que los contienen en todas las columnas:
# No se puede ejecutar más de una vez porque vuelve a separar.

for col in df.columns:
  if col == 'Año' or col == 'Cambio' or col == 'Caracteristicas' or col == 'Plazas':
    df[col] = df[col].apply(lambda x: (',').join(x)) # si está en una lista la info, la separas por comas y lo unes todo, quitandome la lista.

df.head(10)

,Marca,Modelo,Version,Precio,Combustible,Año,Kms,Potencia,Cambio,Vehiculo,Url_foto,categoría,Caracteristicas,Grupo,Puede_comprarse,Plazas,Estado,Centimetros_cubicos
0,Fiat,500,1.2 Lounge 69CV,299,GAS,2018,<p>Coche de ocasión certificado y verificado p...,69,MANUAL,Fiat 500,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,CityCar,,1,True,4,OPPORTUNITY,NaN
1,Peugeot,208,1.2 Puretech Active,319,GAS,2018,<p>Coche de ocasión certificado y verificado p...,82,MANUAL,Peugeot 208,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,Utilitario,"85Cv de potencia,Color disponible - Blanco",1,True,5,OPPORTUNITY,1200.0
2,Ford,Ecosport,1.0T EcoBoost,369,GAS,2019,<p>Coche de ocasión certificado y verificado p...,125,MANUAL,Ford Ecosport,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,B-SUV,,1,True,5,OPPORTUNITY,999.0
3,Seat,Ibiza,1.6 TDI Reference,339,DIESEL,2019,<p>Coche de ocasión certificado y verificado p...,80,MANUAL,Seat Ibiza,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,Utilitario,"115 Cv,5 Plazas,Manual,Diésel,Seminuevo",1,True,5,OPPORTUNITY,1598.0
4,Peugeot,3008,1.5 BlueHDi GT Line,469,DIESEL,2018,<p>Coche de seminuevo certificado y verificado...,130,AUTOMATIC,Peugeot 3008,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,C-SUV,"130 Cv,Acabado GT Line",1,True,5,OPPORTUNITY,1499.0
5,Lexus,UX,2.0 250h,539,HYBRID,2022,None,184,AUTOMATIC,Lexus UX,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,C-SUV,"120 Cv,5 Plazas,Manual,Diésel,Blanco Gris",1,True,5,NEW,2000.0
6,Toyota,C-HR,2.0 180H Advance,569,HYBRID,2022,NaN,184,AUTOMATIC,Toyota C-HR,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,C-SUV,,1,True,5,NEW,1987.0
7,DS,DS7,BlueHDI Performance,629,DIESEL,2021,<p>Coche de ocasión certificado y verificado p...,130,AUTOMATIC,DS DS7,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,D-SUV,,1,True,5,OPPORTUNITY,1199.0
8,Citroen,C3,1.2 Puretech Feel,329,GAS,2020,<p>Coche de ocasión certificado y verificado p...,83,MANUAL,Citroen C3,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,Utilitario,,1,True,5,OPPORTUNITY,1200.0
9,Renault,Megane,E-Tech Equilibre,699,ELECTRIC,2023,None,220,AUTOMATIC,Renault Megane,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,Compacto,,2,True,5,NEW,NaN


---

Necesitamos columna de *kilometraje* de los coches. Vamos a crear dos columnas nuevas en base a columna 'Kms', columna 'Kms_desde' y columna 'Kms_hasta'.

In [ ]:
# Convertimos a string por que da problemas...
df['Kms'] = df['Kms'].astype(str)

for i in df['Kms']:
  print(i)

<p>Coche de ocasión certificado y verificado por Bipi.&nbsp;</p><p><br></p><p>Ventajas de suscribirte a un coche de ocasión de Bipi: más económico, más flexibilidad y sin preocupaciones. El mantenimiento, seguro, ITV e impuestos están incluidos. Nuestros coches pasan revisión y se desinfectan antes de la entrega. Estado: verificados y certificados con un kilometraje entre 30,000 - 100,000 según stock y disponibilidad.</p><p>Combustible: Gasolina y/o GLP.</p>
<p>Coche de ocasión certificado y verificado por Bipi.&nbsp;</p><p><br></p><p>Ventajas de suscribirte a un coche de ocasión de Bipi: más económico, más flexibilidad y sin preocupaciones. El mantenimiento, seguro, ITV e impuestos están incluidos. Nuestros coches pasan revisión y se desinfectan antes de la entrega. Estado: verificados y certificados con un kilometraje entre 20,000 - 50,000 según stock y disponibilidad. Recuerda que siempre podrás cambiar de coche más adelante si tus necesidades cambian</p>
<p>Coche de ocasión certifi

## Usamos REGEX para encontrar kms

In [ ]:
# Encontrar en columna Kms, valores que puedan ser kilometros:

# Primero eliminamos posibles signos de puntuación que separen los números, ejemplo: 20,000
df['Kms'] = df['Kms'].apply(lambda x: x.replace(",",""))
df['Kms'] = df['Kms'].apply(lambda x: x.replace(".",""))
import re

df['Kms_desde'] = 0 # Creamos columnas nuevas que van a almacenar el kilometraje.
df['Kms_hasta'] = 0

for indice in range(len(df['Kms'])):
  patron = r"[1-9][0-9][0][0][0]{1,5}"
  kilometros = re.findall(patron, df['Kms'][indice])
  df['Kms'][indice] = kilometros
  if len(kilometros) == 2:
    df['Kms_desde'][indice] = kilometros[0]
    df['Kms_hasta'][indice] = kilometros[1]
  else:
    df['Kms_desde'][indice] = '0'
    df['Kms_hasta'][indice] = '0'

<ipython-input-236-96dc109af742>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Kms'][indice] = kilometros
<ipython-input-236-96dc109af742>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Kms_desde'][indice] = kilometros[0]
<ipython-input-236-96dc109af742>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Kms_hasta'][indice] = kilometros[1]


In [ ]:
print(f"Únicos para columna 'kms_desde':\n\n{df['Kms_desde'].unique()}\n\n Únicos para columna 'kms_hasta':\n\n {df['Kms_hasta'].unique()}")

Únicos para columna 'kms_desde':

['30000' '20000' '10000' '0' '40000' '70000' '50000' '90000' '60000']

 Únicos para columna 'kms_hasta':

 ['100000' '50000' '40000' '0' '60000' '70000' '30000' '25000' '20000'
 '85000' '80000' '90000']


In [ ]:
# Vamos a arreglar, antes de nada, el valor de un millón que aparece en algunas filas, seguramente será un valor de 100 mil.

df = df.astype({"Kms_desde":"str", "Kms_hasta":"str"})

for indice in range(len(df['Kms_desde'])):
  if df['Kms_desde'][indice] == '1000000':
    df['Kms_desde'][indice] = '100000'

for indice in range(len(df['Kms_hasta'])):
  if df['Kms_hasta'][indice] == '1000000':
    df['Kms_hasta'][indice] = '100000'


Por último vamos a convertir columnas 'Kms_desde' y 'Kms_hasta' a intero.

In [ ]:
# Convertimos columnas nuevas a entero.
# Primeramente eliminamos la ',' que hace de separador de miles.

# df['Kms_desde'] = df['Kms_desde'].apply(lambda x: x.replace(',','').strip())
# df['Kms_hasta'] = df['Kms_hasta'].apply(lambda x: x.replace(',','').strip())

# Convertimos a intero:
df = df.astype({"Kms_desde":"Int64", "Kms_hasta":"Int64"})

In [ ]:
df.dtypes

Marca                   object
Modelo                  object
Version                 object
Precio                   int64
Combustible             object
Año                     object
Kms                     object
Potencia                 int64
Cambio                  object
Vehiculo                object
Url_foto                object
categoría               object
Caracteristicas         object
Grupo                   object
Puede_comprarse           bool
Plazas                  object
Estado                  object
Centimetros_cubicos    float64
Kms_desde                Int64
Kms_hasta                Int64
dtype: object

Reordenamos nuestras columnas en el dataframe y eliminamos columna 'Kms' que ya no aporta información:


In [ ]:
# Reordenamos y quitamos 'Kms':
df = df[['Marca', 'Modelo', 'Version', 'Precio', 'Combustible', 'Año', 'Kms_desde', 'Kms_hasta', 'Potencia', 'Cambio',
        'Vehiculo', 'Url_foto', 'categoría', 'Caracteristicas', 'Grupo', 'Puede_comprarse', 'Plazas', 'Estado',
         'Centimetros_cubicos']]

Convertimos columna 'Año' en datetime, una vez que sus valores no están dentro de una lista:

In [ ]:
# Cambiamos 'Año' a datetime:

df['Año'] = pd.to_datetime(df["Año"], format='%Y')

In [ ]:
# Comprobamos los tipos de datos:
df['Año'].dtypes

dtype('<M8[ns]')

### Valores únicos

In [ ]:
# Una vez arreglado:
# Valores únicos por columna:
  # Comprobamos que está todo correcto.

for col in df.columns:
  print(f" Únicos columna \"{col.upper()}\" ({df[col].nunique()} valores distintos): \n\n {df[col].unique()}")
  print("\n____________________________________________________________________________________________________\n")

 Únicos columna "MARCA" (21 valores distintos): 

 ['Fiat' 'Peugeot' 'Ford' 'Seat' 'Lexus' 'Toyota' 'DS' 'Citroen' 'Renault'
 'Cupra' 'Mercedes-Benz' 'Audi' 'Volkswagen' 'Nissan' 'Opel' 'BMW' 'Kia'
 'Alfa Romeo' 'Hyundai' 'Mitsubishi' 'Dacia']

____________________________________________________________________________________________________

 Únicos columna "MODELO" (47 valores distintos): 

 ['500' '208' 'Ecosport' 'Ibiza' '3008' 'UX' 'C-HR' 'DS7' 'C3' 'Megane'
 '308' 'Formentor' 'GLA' 'Fiesta' 'A1' 'Kuga' 'Polo' 'Clio' 'León'
 'Qashqai' 'Passat' 'Mokka' 'X2' 'Xceed' 'Stelvio' 'Niro' 'Berlingo'
 'Stonic' 'C3 Aircross' 'Clase B' 'Tucson' 'C4' '5008' '2008' 'Sportage'
 'DS3 Crossback' 'Serie 4' 'Ateca' 'Focus' 'Kangoo' 'T-Roc'
 'Eclipse Cross' 'Clase A' 'Spring' 'A3' 'Micra' 'NX']

____________________________________________________________________________________________________

 Únicos columna "VERSION" (49 valores distintos): 

 ['1.2 Lounge 69CV' '1.2 Puretech Active' '1.0T Eco

Eliminamos duplicados, si los hubiese:

In [ ]:
len(df)

50

In [ ]:
# Vemos si exiten filas duplicadas y cuántas:

print(f"Existen {df.duplicated(keep='first').sum()} filas duplicadas, teniendo en cuenta todas las columnas.")
print()
df[df.duplicated()].head(5) # Mostramos los duplicados. No se toma en cuenta la primera aparición de estos.

Existen 0 filas duplicadas, teniendo en cuenta todas las columnas.



,Marca,Modelo,Version,Precio,Combustible,Año,Kms_desde,Kms_hasta,Potencia,Cambio,Vehiculo,Url_foto,categoría,Caracteristicas,Grupo,Puede_comprarse,Plazas,Estado,Centimetros_cubicos


In [ ]:
# Eliminamos las filas duplicadas:

df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False) # argumentos por defecto tambien funciona.

In [ ]:
# Comprobamos:
len(df)

50

In [ ]:
df

,Marca,Modelo,Version,Precio,Combustible,Año,Kms_desde,Kms_hasta,Potencia,Cambio,Vehiculo,Url_foto,categoría,Caracteristicas,Grupo,Puede_comprarse,Plazas,Estado,Centimetros_cubicos
0,Fiat,500,1.2 Lounge 69CV,299,GAS,2018-01-01,30000,100000,69,MANUAL,Fiat 500,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,CityCar,,1,True,4,OPPORTUNITY,NaN
1,Peugeot,208,1.2 Puretech Active,319,GAS,2018-01-01,20000,50000,82,MANUAL,Peugeot 208,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,Utilitario,"85Cv de potencia,Color disponible - Blanco",1,True,5,OPPORTUNITY,1200.0
2,Ford,Ecosport,1.0T EcoBoost,369,GAS,2019-01-01,10000,40000,125,MANUAL,Ford Ecosport,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,B-SUV,,1,True,5,OPPORTUNITY,999.0
3,Seat,Ibiza,1.6 TDI Reference,339,DIESEL,2019-01-01,0,0,80,MANUAL,Seat Ibiza,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,Utilitario,"115 Cv,5 Plazas,Manual,Diésel,Seminuevo",1,True,5,OPPORTUNITY,1598.0
4,Peugeot,3008,1.5 BlueHDi GT Line,469,DIESEL,2018-01-01,20000,60000,130,AUTOMATIC,Peugeot 3008,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,C-SUV,"130 Cv,Acabado GT Line",1,True,5,OPPORTUNITY,1499.0
5,Lexus,UX,2.0 250h,539,HYBRID,2022-01-01,0,0,184,AUTOMATIC,Lexus UX,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,C-SUV,"120 Cv,5 Plazas,Manual,Diésel,Blanco Gris",1,True,5,NEW,2000.0
6,Toyota,C-HR,2.0 180H Advance,569,HYBRID,2022-01-01,0,0,184,AUTOMATIC,Toyota C-HR,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,C-SUV,,1,True,5,NEW,1987.0
7,DS,DS7,BlueHDI Performance,629,DIESEL,2021-01-01,40000,70000,130,AUTOMATIC,DS DS7,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,D-SUV,,1,True,5,OPPORTUNITY,1199.0
8,Citroen,C3,1.2 Puretech Feel,329,GAS,2020-01-01,30000,50000,83,MANUAL,Citroen C3,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,Utilitario,,1,True,5,OPPORTUNITY,1200.0
9,Renault,Megane,E-Tech Equilibre,699,ELECTRIC,2023-01-01,0,0,220,AUTOMATIC,Renault Megane,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,Compacto,,2,True,5,NEW,NaN


# cambiamos 'Año' por 'Antiguedad' y demás columnas sin acentos

In [ ]:
df.columns

Index(['Marca', 'Modelo', 'Version', 'Precio', 'Combustible', 'Año',
       'Kms_desde', 'Kms_hasta', 'Potencia', 'Cambio', 'Vehiculo', 'Url_foto',
       'categoría', 'Caracteristicas', 'Grupo', 'Puede_comprarse', 'Plazas',
       'Estado', 'Centimetros_cubicos'],
      dtype='object')

In [ ]:
df = df.rename(columns={'Año': 'Antiguedad'})

df = df.rename(columns={'categoría':'categoria'})
df.columns

Index(['Marca', 'Modelo', 'Version', 'Precio', 'Combustible', 'Antiguedad',
       'Kms_desde', 'Kms_hasta', 'Potencia', 'Cambio', 'Vehiculo', 'Url_foto',
       'categoria', 'Caracteristicas', 'Grupo', 'Puede_comprarse', 'Plazas',
       'Estado', 'Centimetros_cubicos'],
      dtype='object')

## 4 GUARDAMOS DF, .CSV DESCARGABLE:

In [ ]:
# Convertimos el df en archivo csv y descargable:

df.to_csv('CSV_BIPICAR_LIMPIO_Proyecto_Jupiter1.csv', index=False)


In [ ]:
df.head(1)

,Marca,Modelo,Version,Precio,Combustible,Antiguedad,Kms_desde,Kms_hasta,Potencia,Cambio,Vehiculo,Url_foto,categoria,Caracteristicas,Grupo,Puede_comprarse,Plazas,Estado,Centimetros_cubicos
0,Fiat,500,1.2 Lounge 69CV,299,GAS,2018-01-01,30000,100000,69,MANUAL,Fiat 500,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,CityCar,,1,True,4,OPPORTUNITY,NaN


## 5 Probando a extraer dfs por días y concatenar para ir extrayendo coches sin duplicados:

In [ ]:
df1 = pd.read_csv('CSV_BIPICAR_LIMPIO_Proyecto_Jupiter_Completo23.csv') # el que ya tenemos en pc

df4 = pd.read_csv('CSV_BIPICAR_LIMPIO_Proyecto_Jupiter1.csv') # el que acabamos de sacar

In [ ]:
df1

,Marca,Modelo,Version,Precio,Combustible,Antiguedad,Kms_desde,Kms_hasta,Potencia,Cambio,Vehiculo,Url_foto,categoria,Caracteristicas,Grupo,Puede_comprarse,Plazas,Estado,Centimetros_cubicos
0,Fiat,500,Hybrid Dolcevita,325.0,HYBRID,2021-01-01,20000.0,40000.0,70,MANUAL,Fiat 500,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,CityCarnan,NaN,2,True,4,OPPORTUNITY,999.0
1,Citroen,C3,1.2 Puretech Feel,329.0,GAS,2020-01-01,30000.0,50000.0,83,MANUAL,Citroen C3,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,Utilitarionan,NaN,1,True,5,OPPORTUNITY,1200.0
2,Toyota,C-HR,1.8 125H,449.0,HYBRID,2019-01-01,20000.0,100000.0,122,AUTOMATIC,Toyota C-HR,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,C-SUVnan,NaN,1,True,5,OPPORTUNITY,1798.0
3,Lexus,UX,2.0 250h,625.0,HYBRID,2022-01-01,0.0,0.0,184,AUTOMATIC,Lexus UX,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,C-SUVnan,"120 Cv,5 Plazas,Manual,Diesel,Blanco Gris",1,True,5,NEW,2000.0
4,Peugeot,3008,1.5L BlueHDi S&S GT,459.0,DIESEL,2018-01-01,50000.0,95000.0,130,MANUAL,Peugeot 3008,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,C-SUVnan,"130 Cv,Acabado GT Line",1,True,5,OPPORTUNITY,1200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,Seat,Ibiza,1.6 TDI Reference,335.0,DIESEL,2019-01-01,0.0,0.0,80,MANUAL,Seat Ibiza,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,nanUtilitario,"115 Cv,5 Plazas,Manual,Diesel,Seminuevo",1,True,5,OPPORTUNITY,1598.0
282,Ford,Focus,1.5 TDCi Trend,365.0,DIESEL,2018-01-01,50000.0,80000.0,95,MANUAL,Ford Focus,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,nanCompacto,"120 Cv,5 Plazas,Manual,Diesel,Blanco,Seminuevo",1,True,5,OPPORTUNITY,0.0
283,Mercedes-Benz,Clase A,200d,479.0,DIESEL,2019-01-01,20000.0,80000.0,150,AUTOMATIC,Mercedes-Benz Clase A,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,nanCompacto,NaN,1,True,5,OPPORTUNITY,2000.0
284,Renault,Clio,1.0 equilibre TCe,349.0,GAS,2022-01-01,0.0,0.0,90,MANUAL,Renault Clio,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,nanUtilitario,"85Cv de potencia,Color disponible Blanco,Vehic...",4,True,5,OPPORTUNITY,1000.0


In [ ]:
if 'categoría' in df1.columns and 'categoria' in df1.columns:
  df1['categoría'] = df1['categoría'].astype(str).fillna('')
  df1['categoria'] = df1['categoria'].astype(str).fillna('')

  df1['categoría'] = df1['categoría'] + df1['categoria']
  df1.drop('categoria', inplace=True, axis=1)
  df1 = df1.rename(columns = {'categoría':'categoria'})



In [ ]:
df1['categoria'] = df1['categoria'].apply(lambda x: x.replace('nan',''))

In [ ]:
# eliminamos columnas antiguedad que sobran y año si las tiene:

# df1 = df1.drop(['Antiguedad.1', 'Antiguedad.2'], axis=1)


In [ ]:
df_all = pd.concat([df1, df4])

In [ ]:
df_all.shape

(336, 19)

In [ ]:
df_all.reset_index(drop=True, inplace=True)

In [ ]:
df_all.duplicated().sum()
 # con los dos primeros dfs hay 40 duplicados

56

In [ ]:
df_all.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

In [ ]:
df_all.shape
# con los dos primeros dfs hay 46 filas.

(280, 19)

In [ ]:
df_all.reset_index(drop=True, inplace=True)

# Quitamos acentos para evitar errores en sql

In [ ]:
for columna in df_all:
  if df_all[columna].dtypes == object:
    df_all[columna] = df_all[columna].astype(str)
    print(columna)
  # if columna == 'Marca' or columna == 'Modelo' or columna == 'Version' or columna == 'Combustible' or columna == 'Cambio' or columna == 'Color':
    df_all[columna] = df_all[columna].apply(lambda x: x.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u').replace('Á','A').replace('É','E').replace('Í','I').replace('Ó','O').replace('Ú','U'))



Marca
Modelo
Version
Combustible
Antiguedad
Cambio
Vehiculo
Url_foto
categoria
Caracteristicas
Plazas
Estado


# Quitamos NaN que da error en SQL y los nan cadena por ''

In [ ]:
for columna in df_all:
  if df_all[columna].dtypes == int or df_all[columna].dtypes == float:
    print(columna)
    df_all[columna].loc[df_all[df_all[columna].isnull() == True].index] = 0

# df["doors"].loc[df[df["doors"] == 2].index] = 3

Precio
Kms_desde
Kms_hasta
Potencia
Grupo
Centimetros_cubicos


<ipython-input-270-34de8c74812c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all[columna].loc[df_all[df_all[columna].isnull() == True].index] = 0


In [ ]:
# para las columnas que no sean objeto, guardamos su tipo, convertimos en str, quitamos NaN, y volvemos a su tipo!!!

for columna in df_all:
  tipo = df_all[columna].dtypes

  df_all[columna] = df_all[columna].astype(str)
  df_all[columna] = df_all[columna].apply(lambda x: x.replace('nan', ''))

  df_all[columna] = df_all[columna].astype(tipo)
  print(columna)
  # if columna == 'Marca' or columna == 'Modelo' or columna == 'Version' or columna == 'Combustible' or columna == 'Cambio' or columna == 'Color':
  # df_all[columna] = df_all[columna].apply(lambda x: x.replace('NaN', 0))


Marca
Modelo
Version
Precio
Combustible
Antiguedad
Kms_desde
Kms_hasta
Potencia
Cambio
Vehiculo
Url_foto
categoria
Caracteristicas
Grupo
Puede_comprarse
Plazas
Estado
Centimetros_cubicos


In [ ]:
df_all.head(1)

,Marca,Modelo,Version,Precio,Combustible,Antiguedad,Kms_desde,Kms_hasta,Potencia,Cambio,Vehiculo,Url_foto,categoria,Caracteristicas,Grupo,Puede_comprarse,Plazas,Estado,Centimetros_cubicos
0,Fiat,500,Hybrid Dolcevita,325.0,HYBRID,2021-01-01,20000.0,40000.0,70,MANUAL,Fiat 500,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,CityCar,,2,True,4,OPPORTUNITY,999.0


# Guardamos descargable final

In [ ]:
df_all.to_csv('CSV_BIPICAR_LIMPIO_Proyecto_Jupiter_Definitivo.csv', index=False)

In [ ]:
df_all.shape

(280, 19)

In [ ]:
df_all

,Marca,Modelo,Version,Precio,Combustible,Antiguedad,Kms_desde,Kms_hasta,Potencia,Cambio,Vehiculo,Url_foto,categoria,Caracteristicas,Grupo,Puede_comprarse,Plazas,Estado,Centimetros_cubicos
0,Fiat,500,Hybrid Dolcevita,325.0,HYBRID,2021-01-01,20000.0,40000.0,70,MANUAL,Fiat 500,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,CityCar,,2,True,4,OPPORTUNITY,999.0
1,Citroen,C3,1.2 Puretech Feel,329.0,GAS,2020-01-01,30000.0,50000.0,83,MANUAL,Citroen C3,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,Utilitario,,1,True,5,OPPORTUNITY,1200.0
2,Toyota,C-HR,1.8 125H,449.0,HYBRID,2019-01-01,20000.0,100000.0,122,AUTOMATIC,Toyota C-HR,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,C-SUV,,1,True,5,OPPORTUNITY,1798.0
3,Lexus,UX,2.0 250h,625.0,HYBRID,2022-01-01,0.0,0.0,184,AUTOMATIC,Lexus UX,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,C-SUV,"120 Cv,5 Plazas,Manual,Diesel,Blanco Gris",1,True,5,NEW,2000.0
4,Peugeot,3008,1.5L BlueHDi S&S GT,459.0,DIESEL,2018-01-01,50000.0,95000.0,130,MANUAL,Peugeot 3008,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,C-SUV,"130 Cv,Acabado GT Line",1,True,5,OPPORTUNITY,1200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,Mercedes-Benz,Clase A,180d,565.0,DIESEL,2019-01-01,10000.0,30000.0,116,AUTOMATIC,Mercedes-Benz Clase A,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,Compacto,,1,True,5,OPPORTUNITY,1950.0
276,Dacia,Spring,sharing one electric,395.0,ELECTRIC,2021-01-01,0.0,0.0,45,AUTOMATIC,Dacia Spring,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,A-SUV,,1,True,4,OPPORTUNITY,0.0
277,Audi,A3,30 TFSI Design,439.0,GAS,2018-01-01,20000.0,50000.0,150,MANUAL,Audi A3,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,Compacto,"Gasolina,116 Cv,5 Plazas,Manual,S Line",1,True,5,OPPORTUNITY,1598.0
278,Nissan,Micra,1.0 IG-T Acenta,369.0,GAS,2019-01-01,10000.0,70000.0,100,MANUAL,Nissan Micra,https://bipi-pro.s3.eu-west-1.amazonaws.com/sp...,Utilitario,"Acabado Acenta,Disponible en gasolina o diesel...",1,True,5,OPPORTUNITY,1000.0


In [ ]:
df.dtypes

Marca                          object
Modelo                         object
Version                        object
Precio                          int64
Combustible                    object
Antiguedad             datetime64[ns]
Kms_desde                       Int64
Kms_hasta                       Int64
Potencia                        int64
Cambio                         object
Vehiculo                       object
Url_foto                       object
categoria                      object
Caracteristicas                object
Grupo                          object
Puede_comprarse                  bool
Plazas                         object
Estado                         object
Centimetros_cubicos           float64
dtype: object